In [1]:
import numpy as np
from sklearn import svm

# Contruct the word embeddings using GloVe algorithm

In [2]:
#!/usr/bin/env python3
from scipy.sparse import *
import numpy as np
import pickle
import random


print("loading cooccurrence matrix")
with open('cooc.pkl', 'rb') as f:
    cooc = pickle.load(f)
print("{} nonzero entries".format(cooc.nnz))

nmax = 100
print("using nmax =", nmax, ", cooc.max() =", cooc.max())

print("initializing embeddings")
embedding_dim = 20
xs = np.random.normal(size=(cooc.shape[0], embedding_dim))
ys = np.random.normal(size=(cooc.shape[1], embedding_dim))

eta = 0.001
alpha = 3 / 4

epochs = 10

for epoch in range(epochs):
    print("epoch {}".format(epoch))
    for ix, jy, n in zip(cooc.row, cooc.col, cooc.data):
        logn = np.log(n)
        fn = min(1.0, (n / nmax) ** alpha)
        x, y = xs[ix, :], ys[jy, :]
        scale = 2 * eta * fn * (logn - np.dot(x, y))
        xs[ix, :] += scale * y
        ys[jy, :] += scale * x
np.save('embeddings', xs)



loading cooccurrence matrix
6496907 nonzero entries
using nmax = 100 , cooc.max() = 207302
initializing embeddings
epoch 0
epoch 1
epoch 2
epoch 3
epoch 4
epoch 5
epoch 6
epoch 7
epoch 8
epoch 9


In [3]:
word_embeddings = np.load("embeddings.npy")
word_embeddings.shape

(21161, 20)

Now we have a representation for each word as a vector of dimension 20.

We will load each tweet and construct a representation for this tweet as a mean of the vector of each word contained in the tweet.

In [4]:
words_list = []
f = open("vocab_cut.txt", "r")
for l in f.readlines():
    l = l.strip()
    words_list.append(l)
words_list = np.array(words_list)
f.close()

In [5]:
np.argmax(words_list=="paul")

2133

In [7]:
tweets_pos_txt = []
f = open("Datasets/twitter-datasets/train_pos.txt")
for l in f.readlines():
    tweets_pos_txt.append(l.strip())
tweets_pos_txt = np.array(tweets_pos_txt)
f.close()
tweets_pos_vec = []
for tw in tweets_pos_txt:
    words_in_tweet = tw.split(" ")
    acc = np.zeros(word_embeddings[0].shape[0])
    for w in words_in_tweet:
        vec = word_embeddings[np.argmax(words_list==w)]
        acc += vec
    acc = acc/len(words_in_tweet)
    tweets_pos_vec.append(acc)
tweets_pos_vec = np.array(tweets_pos_vec)

In [9]:
tweets_neg_txt = []
f = open("Datasets/twitter-datasets/train_neg.txt")
for l in f.readlines():
    tweets_neg_txt.append(l.strip())
tweets_neg_txt = np.array(tweets_neg_txt)
f.close()
tweets_neg_vec = []
for tw in tweets_neg_txt:
    words_in_tweet = tw.split(" ")
    acc = np.zeros(word_embeddings[0].shape[0])
    for w in words_in_tweet:
        vec = word_embeddings[np.argmax(words_list==w)]
        acc += vec
    acc = acc/len(words_in_tweet)
    tweets_neg_vec.append(acc)
tweets_neg_vec = np.array(tweets_neg_vec)

In [10]:
train_data = np.concatenate((tweets_pos_vec, tweets_neg_vec))
train_y = np.concatenate((np.ones((tweets_pos_vec.shape[0])), -1*np.ones((tweets_pos_vec.shape[0]))))

# Try with simple SVM

In [ ]:
train_y[train_y == -1] = 0
clf = svm.SVC(gamma='scale')
clf.fit(train_data, train_y)

Load one tweet for test set to try the model